# DUMMY

* https://github.com/unslothai/unsloth
* https://docs.unsloth.ai/get-started/installing-+-updating/windows-installation  METHOD#2 (CC,CXX, CL path)

* conda deactivate
* conda remove --name env_llm_qlora --all -y

Install cudatoolkit
* conda create --name env_llm_qlora python=3.11 pytorch-cuda=12.1 pytorch cudatoolkit -c pytorch -c nvidia -y
* conda activate env_llm_qlora


* ####python -m pip install xformers | conda install -c conda-forge xformers |

* python -m pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
* [Blackwell] python -m pip install -U xformers --index-url https://download.pytorch.org/whl/nightly/cu128 (python -m pip uninstall xformers -y)

python -m pip install "unsloth[windows] @ git+https://github.com/unslothai/unsloth.git"
python -m pip install --no-deps trl peft accelerate bitsandbytes
python -m pip install ipywidgets ipykernel
python -m pip install jmespath
python -m pip install --upgrade accelerate
python -m pip install tensorboard

[Blackwell]
python -m pip uninstall torch torchvision torchaudio -y
python -m pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128
python -m pip install --no-deps xformers==0.0.29.post3

Patch: num_proc = 1

# Imports

In [1]:
import os
import torch
import json

# Unsloth always first as import, otherwise will create some inconsistent errors
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template, train_on_responses_only

from trl import SFTTrainer
from datasets import Dataset
from threading import Thread
from transformers import TrainingArguments, DataCollatorForSeq2Seq, TextIteratorStreamer, TextStreamer


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
cuda_available = torch.cuda.is_available()
print(f"CUDA disponible: {cuda_available}")

# Mostrar el dispositivo actual
if cuda_available:
    device = torch.device("cuda")
    print(f"Dispositivo actual: {torch.cuda.get_device_name(0)}")
    print(f"Cantidad de GPUs disponibles: {torch.cuda.device_count()}")
else:
    device = torch.device("cpu")
    print("Se está utilizando la CPU.")

CUDA disponible: True
Dispositivo actual: NVIDIA GeForce RTX 5070 Ti
Cantidad de GPUs disponibles: 2


In [3]:
# Definimos parámetros importantes
max_seq_length = 512         # Máxima longitud de secuencia (ajústala si necesitas más contexto)
dtype = None                 # Puede ser especificado, en este ejemplo lo dejamos por defecto
load_in_4bit = True          # Usamos 4bit para optimizar uso de VRAM

print("\nCargando el modelo base y el tokenizador...")
# Cargamos el modelo preentrenado de Unsloth. Se utilizará el modelo LLaMA-3.2-3B-bnb-4bit.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
print("Modelo y tokenizador cargados con éxito.")

# Configuramos el modelo para LoRA. Se inyectan matrices LoRA en las proyecciones relevantes.
print("\nAplicando configuración LoRA al modelo...")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Puede ajustarse; valores sugeridos: 8, 16, 32, 64, etc.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # , "gate_proj", "up_proj", "down_proj"
    lora_alpha=16,
    lora_dropout=0,   # Se recomienda 0 para optimización
    bias="none",
    use_gradient_checkpointing="unsloth",  # Para manejar contextos largos
    random_state=4242,
    use_rslora=False,  # Desactivar RSLORA si no se requiere
    loftq_config=None,
)
print("Configuración LoRA aplicada.")


Cargando el modelo base y el tokenizador...
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 5070 Ti. Num GPUs = 2. Max memory: 15.92 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Modelo y tokenizador cargados con éxito.

Aplicando configuración LoRA al modelo...


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.4.7 patched 28 layers with 28 QKV layers, 28 O layers and 0 MLP layers.


Configuración LoRA aplicada.


| Módulo           | ¿Cuándo incluirlo en `target_modules`?                                             | Notas adicionales |
|------------------|------------------------------------------------------------------------------------|-------------------|
| `q_proj`         | ✅ **Siempre**                                                                     | La matriz Query determina cómo se consulta la atención, esencial para aprendizaje de contextos específicos y conversaciones. |
| `k_proj`         | ✅ **Siempre**                                                                     | La matriz Key determina cómo se organizan las representaciones internas del contexto. Necesaria para capturar patrones semánticos. |
| `v_proj`         | ✅ **Siempre**                                                                     | La matriz Value representa el valor real o contenido semántico; fundamental para generar respuestas coherentes y precisas. |
| `o_proj`         | ✅ **Siempre o casi siempre**                                                       | Proyección de salida final en atención; afecta directamente la calidad y claridad de las respuestas. Suele mejorar la coherencia. |
| `gate_proj`      | 🔶 **A veces** (Útil especialmente en tareas complejas o textos largos)            | Controla la activación en capas FFN; ayuda en tareas con mucha información o patrones complejos. Puede mejorar calidad, pero añade memoria y tiempo. |
| `up_proj`        | 🔶 **A veces** (Útil en datasets grandes o muy diversos)                           | Expande la dimensión interna en FFN. Agregarla mejora ligeramente capacidad expresiva pero consume más memoria. |
| `down_proj`      | 🔶 **A veces** (Útil junto con `up_proj` en tareas complejas o datasets variados)  | Reduce la dimensión en FFN, actúa junto con `up_proj`. Se recomienda si se usa `up_proj`. |

### 📌 **Recomendaciones prácticas rápidas:**

- **Casos estándar (p. ej. chatbots, asistentes):**
  ```python
  target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
  ```

- **Casos avanzados (textos largos, tareas complejas, fine-tuning intensivo):**
  ```python
  target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
  ```

In [4]:
# Si el tokenizador no tiene definido un token de padding, lo asignamos a uno de los tokens reservados.
if tokenizer.pad_token is None:
    # Elegimos usar <|reserved_special_token_0|> como token de padding
    tokenizer.pad_token = "<|reserved_special_token_0|>"
    print("Se asignó <|reserved_special_token_0|> como token PAD.")

# Verificamos que el modelo tenga configurado el pad_token_id.
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.pad_token_id
    print(f"Se configuró el pad_token_id del modelo a: {model.config.pad_token_id}")

# Asegurarse de que el token EOS esté configurado (por lo general es <|end_of_text|>)
if not hasattr(tokenizer, "eos_token") or tokenizer.eos_token is None:
    # En muchos modelos, el EOS es "<|end_of_text|>"; ajusta según tu modelo.
    tokenizer.eos_token = "<|end_of_text|>"
    print("Se asignó <|end_of_text|> como token EOS.")

print("Configuración de tokens especiales completada.")

Configuración de tokens especiales completada.


In [5]:
print(tokenizer.pad_token)
print(tokenizer.eos_token)

<|finetune_right_pad_id|>
<|end_of_text|>


You typically don't have to manually add <|end_of_text|> in the conversation formatting or the dataset itself. The token <|end_of_text|> is added automatically by the tokenizer or the generation logic to indicate the end of generation.

In [6]:
# Definimos el system prompt para fijar la personalidad del asistente.
system_prompt = {
    "role": "system",
    "content": ("Eres una streamer divertida, carismática y auténtica. "
                "Siempre respondes con humor, espontaneidad y energía positiva. "
                "Interactúas de forma cercana con tu audiencia, utilizando expresiones coloquiales y reacciones genuinas.")
}

# Ruta del dataset (formato JSONL, donde cada línea es una conversación)
dataset_path = "./temp/dataset.jsonl"

print("\nLeyendo y procesando el dataset de conversaciones...")
data = []
# Abrimos el archivo y procesamos cada línea
with open(dataset_path, "r", encoding="utf8") as f:
    for line in f:
        conversation = json.loads(line)
        # Verificamos que la conversación sea una lista y que cada mensaje tenga 'role' y 'content'
        if isinstance(conversation, list):
            # Insertamos el system prompt al inicio para anclar la personalidad
            conversation.insert(0, system_prompt)
            # Realizamos una comprobación básica: cada mensaje debe ser un dict con 'role' y 'content'
            valid = all(isinstance(m, dict) and "role" in m and "content" in m for m in conversation)
            assert all("role" in m and "content" in m for m in conversation), "Conversación mal formada."
        
            if not valid:
                print("Advertencia: Se encontró una conversación con formato inesperado. Revisar:")
                print(conversation)
            data.append({"conversation": conversation})
        else:
            print("Línea con formato inesperado:", line)

print(f"Total de ejemplos cargados: {len(data)}")
print(f"Sample: {data[0]}")

# Creamos el objeto dataset de Hugging Face
dataset = Dataset.from_list(data)
print("Dataset creado correctamente.")

# Imprimir un ejemplo del dataset original para validación
print("\nEjemplo de conversación original (sin formatear):")
print(dataset[0]["conversation"])


Leyendo y procesando el dataset de conversaciones...
Total de ejemplos cargados: 241
Sample: {'conversation': [{'role': 'system', 'content': 'Eres una streamer divertida, carismática y auténtica. Siempre respondes con humor, espontaneidad y energía positiva. Interactúas de forma cercana con tu audiencia, utilizando expresiones coloquiales y reacciones genuinas.'}, {'content': '¿Qué tipo de gaseosa te gusta?', 'role': 'user'}, {'content': 'Gaseosa de naranja, obviamente. Es como el jugo de la vida pero con burbujas.', 'role': 'assistant'}]}
Dataset creado correctamente.

Ejemplo de conversación original (sin formatear):
[{'content': 'Eres una streamer divertida, carismática y auténtica. Siempre respondes con humor, espontaneidad y energía positiva. Interactúas de forma cercana con tu audiencia, utilizando expresiones coloquiales y reacciones genuinas.', 'role': 'system'}, {'content': '¿Qué tipo de gaseosa te gusta?', 'role': 'user'}, {'content': 'Gaseosa de naranja, obviamente. Es como

In [7]:
print("\nConfigurando la plantilla de chat para el tokenizador...")
# Inicializamos el tokenizador con la plantilla "llama-3.2"
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.2")
print("Plantilla de chat aplicada.")

# Función para formatear cada conversación utilizando la plantilla
def formatting_prompts_func(examples):
    texts = []
    for convo in examples["conversation"]:
        # Aplicamos la plantilla de chat a cada conversación
        formatted = tokenizer.apply_chat_template(
            convo,
            tokenize=False,            # Queremos el texto legible para depuración
            add_generation_prompt=False  # No agregamos prompt adicional en esta etapa
        )
        texts.append(formatted)
    # Se devuelve un diccionario con la clave "text" para el dataset
    return {"text": texts}

# Verificamos manualmente el formateo en un ejemplo específico
example_index = 0  # Cambia el índice según convenga
example_convo = dataset[example_index]["conversation"]
print("\nEjemplo de conversación (raw):")
print(example_convo)

# Aplicamos la plantilla al ejemplo y mostramos el resultado
formatted_example = tokenizer.apply_chat_template(example_convo, tokenize=False, add_generation_prompt=False)
print("\nEjemplo de conversación formateada:")
print(formatted_example)

# Mapeamos el dataset completo para aplicar el formateo
print("\nAplicando formateo a todo el dataset...")
dataset = dataset.map(formatting_prompts_func, batched=True)
print("Formateo aplicado. Mostrando ejemplo final del dataset:")
print(dataset[example_index]["text"])


Configurando la plantilla de chat para el tokenizador...
Plantilla de chat aplicada.

Ejemplo de conversación (raw):
[{'content': 'Eres una streamer divertida, carismática y auténtica. Siempre respondes con humor, espontaneidad y energía positiva. Interactúas de forma cercana con tu audiencia, utilizando expresiones coloquiales y reacciones genuinas.', 'role': 'system'}, {'content': '¿Qué tipo de gaseosa te gusta?', 'role': 'user'}, {'content': 'Gaseosa de naranja, obviamente. Es como el jugo de la vida pero con burbujas.', 'role': 'assistant'}]

Ejemplo de conversación formateada:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

Eres una streamer divertida, carismática y auténtica. Siempre respondes con humor, espontaneidad y energía positiva. Interactúas de forma cercana con tu audiencia, utilizando expresiones coloquiales y reacciones genuinas.<|eot_id|><|start_header_id|>user<|end_header_id|>

¿Qué tipo de

Map:   0%|          | 0/241 [00:00<?, ? examples/s]

Formateo aplicado. Mostrando ejemplo final del dataset:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

Eres una streamer divertida, carismática y auténtica. Siempre respondes con humor, espontaneidad y energía positiva. Interactúas de forma cercana con tu audiencia, utilizando expresiones coloquiales y reacciones genuinas.<|eot_id|><|start_header_id|>user<|end_header_id|>

¿Qué tipo de gaseosa te gusta?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Gaseosa de naranja, obviamente. Es como el jugo de la vida pero con burbujas.<|eot_id|>


The token <|eot_id|> is a special marker defined explicitly by the chat template (llama-3.2) to separate segments of conversation internally. It represents the end of turn (End-Of-Turn) in conversations, which can be the end of system, user, or assistant messages.

In [8]:
# ---------------------------------------------------------------------------
# 6. Definir funciones de generación para pruebas (One-shot y Streaming)
# ---------------------------------------------------------------------------
def response_normal(model, tokenizer, messages, temperature=0.7, max_new_tokens=64, skip_special_tokens=False):
    """
    Función para obtener la respuesta completa (one-shot) sin streaming.
    Se habilita la inferencia optimizada y se generan tokens.
    """
    # Optimización de inferencia con Unsloth
    FastLanguageModel.for_inference(model)
    
    # Preparar las entradas usando la plantilla de chat
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,  # Es necesario para la generación
        return_tensors="pt",
    ).to(device)
    
    # Generación de respuesta (one-shot)
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=max_new_tokens,
        use_cache=True,
        temperature=0.7,              # Lower this to reduce randomness
        # repetition_penalty=1.1,       # Avoid repetitive outputs
        # min_p=0.1
    )
    
    # Decodificar y retornar la respuesta, omitiendo tokens especiales
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=skip_special_tokens)
    return decoded

# Función opcional para generación en streaming (comentada en este ejemplo)
def response_stream(model, tokenizer, messages):
    """
    Función para generación de respuesta en streaming.
    Permite ver los tokens generados progresivamente.
    """
    FastLanguageModel.for_inference(model)
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(device)
    
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)
    thread = Thread(target=model.generate, kwargs={
        "input_ids": inputs,
        "streamer": streamer,
        "max_new_tokens": 128,
        "use_cache": True,
        "temperature": 1.0,
        "min_p": 0.1
    })
    thread.start()
    for new_text in streamer:
        print(new_text, end="", flush=True)

In [ ]:
# ------------------------------------------------------------------------------
# START behaviour
# ------------------------------------------------------------------------------
# Get first message from a conversation:
conversation_user_message = list(filter(lambda x: x['role'] == 'user', dataset[5]["conversation"]))[0]['content'] # first message
messages = [
    system_prompt,
    {"role": "user", "content": conversation_user_message}
]

# Don't run because it will change to inference mode, but if you run remember to restore the model to previous state
# print(messages)
# # 1) Generación normal (sin streaming)
# print("BY ONE SHOT:")
# llm_response = response_normal(model, tokenizer, messages)
# print(llm_response[-1])

# 2) Generación en streaming
# print("\nBY STREAMING:")
# for chunk in response_stream(model, tokenizer, messages):
#     print(chunk, end="", flush=True)

The model hasn't been fine-tuned yet, so it's just responding based on random weights from LoRA adapters.

Generation parameters (like temperature or repetition_penalty) aren't correctly configured.

To create a comparison of before and afer: Run inference with the original pretrained base model without LoRA adapters. You should see generic but coherent outputs.

In [ ]:
if False:
    # For retrain
    # Reload Model & Tokenizer (as above)
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="./models/qlora_mirai_v0-3B",
        max_seq_length=512,
        load_in_4bit=True
    )

    model = FastLanguageModel.get_peft_model(
        model,
        r=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_alpha=16,
        random_state=42,
    )

In [9]:
# ---------------------------------------------------------------------------
# 7. Configuración del Trainer para Fine-Tuning con QLoRA
# ---------------------------------------------------------------------------
print("\nConfigurando el entrenamiento con QLoRA usando SFTTrainer...")

# Creamos el objeto de entrenamiento con SFTTrainer de TRL (usado por Unsloth)
trainer = None

# Configuramos los argumentos de entrenamiento con hiperparámetros recomendados.
training_args = TrainingArguments(
    per_device_train_batch_size=2,             # Batch pequeño debido a la cantidad de parámetros
    gradient_accumulation_steps=4,             # Acumulamos gradientes para simular un batch más grande
    warmup_steps=10,                            # Pasos de warmup
    num_train_epochs=2,                        # Set this for 1 full training run.
    # max_steps=60,                            # Número total de pasos (ajusta según el tamaño de tu dataset)
    learning_rate=1e-4,                        # Tasa de aprendizaje
    fp16=not is_bfloat16_supported(),          # Usamos fp16 si no se soporta bf16
    bf16=is_bfloat16_supported(),              # bf16 si es soportado por la GPU
    logging_steps=2,                          # Registro cada 10 pasos
    optim="adamw_8bit",                        # Optimizador eficiente en memoria
    weight_decay=0.01,                         # Regularización
    lr_scheduler_type="linear",                # Scheduler lineal
    seed=4242,                                 # Fijamos la semilla para reproducibilidad
    output_dir="./outputs",                    # Directorio de salida para guardar checkpoints
    report_to="tensorboard",                          # Deshabilitamos reportes externos (ej. WandB)
)

print("Argumentos de entrenamiento configurados:")
print(training_args)

# Creamos el DataCollator para secuencias (ajustado para seq2seq)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

# Inicializamos el Trainer con el modelo, tokenizador, dataset, y data collator
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,          # El dataset ya formateado tiene la columna "text"
    dataset_text_field="text",      # Indicamos que la entrada es la clave "text"
    max_seq_length=max_seq_length,
    data_collator=data_collator,
    dataset_num_proc=1,             # Número de procesos para mapear (ajusta según tu CPU)
    packing=False,                  # Si las secuencias son cortas, packing puede acelerar el entrenamiento
    args=training_args,
)

print("SFTTrainer configurado correctamente.")

# Aplicamos la función train_on_responses_only para entrenar solo en las respuestas del asistente.
# Es fundamental que se especifiquen correctamente los delimitadores según la plantilla usada.
print("\nAplicando train_on_responses_only para enmascarar las partes de usuario...")
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",  # Delimitador para instrucciones (usuario)
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",  # Delimitador para respuestas (asistente)
)
print("Configuración de enmascaramiento aplicada.")


Configurando el entrenamiento con QLoRA usando SFTTrainer...
Argumentos de entrenamiento configurados:
TrainingArguments(
_n_gpu=2,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,


Unsloth: Tokenizing ["text"]:   0%|          | 0/241 [00:00<?, ? examples/s]

SFTTrainer configurado correctamente.

Aplicando train_on_responses_only para enmascarar las partes de usuario...


Map (num_proc=24):   0%|          | 0/241 [00:00<?, ? examples/s]

Configuración de enmascaramiento aplicada.


We verify masking is actually done:

In [10]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nEres una streamer divertida, carismática y auténtica. Siempre respondes con humor, espontaneidad y energía positiva. Interactúas de forma cercana con tu audiencia, utilizando expresiones coloquiales y reacciones genuinas.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n¿Prefieres series o películas?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nPrefiero series, así tengo excusa para procrastinar varios días seguidos.<|eot_id|>'

In [ ]:
# You should not see the system or user part
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                        Prefiero series, así tengo excusa para procrastinar varios días seguidos.<|eot_id|>'

: 

In [ ]:
# ---------------------------------------------------------------------------
print("\nIniciando el entrenamiento...")
trainer_stats = trainer.train()
print("\nEntrenamiento completado. Estadísticas:")
print(trainer_stats)


Iniciando el entrenamiento...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 241 | Num Epochs = 2 | Total steps = 30
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 9,175,040/3,000,000,000 (0.31% trained)


In [ ]:
# Definimos la ruta para guardar el modelo fine-tuneado
output_model_dir = "./models/qlora_mirai_v0-3B"
os.makedirs(output_model_dir, exist_ok=True)

print("Guardando el modelo y el tokenizador en:", output_model_dir)
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)
print("Modelo y tokenizador guardados correctamente.")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [ ]:
if True:
    # For new loading
    from unsloth import FastLanguageModel
    model_inference, tokenizer_inference = FastLanguageModel.from_pretrained(
        model_name = output_model_dir, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model_inference)

    test_messages = [
        system_prompt,
        {"role": "user", "content": "Rivers, ¿qué opinas del nuevo parche del LoL?"}
    ]

    inputs = tokenizer_inference.apply_chat_template(
        test_messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(device)

    from transformers import TextStreamer
    print("ANSWER:\n")
    text_streamer = TextStreamer(tokenizer_inference, skip_prompt=True)

    _ = model_inference.generate(
        input_ids=inputs,
        streamer=text_streamer,
        max_new_tokens=128,
        temperature=0.6,
        top_p=0.9,
        repetition_penalty=1.1,
        eos_token_id=tokenizer_inference.eos_token_id,  # Esto evita tokens especiales raros
        pad_token_id=tokenizer_inference.pad_token_id,
    )

In [ ]:
_ = model_inference.generate(
    input_ids=inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    temperature=0.6,
    top_p=0.3,
    repetition_penalty=1.3,
    eos_token_id=tokenizer_inference.eos_token_id,  # Esto evita tokens especiales raros
    pad_token_id=tokenizer_inference.pad_token_id,
)

TODO: later
* Long Term memory
* Coyote time in interaction
* Filter: https://github.com/neurokitti/AIRIS-VtuberAI
* Emotions
* Character / Personality
* Input centralizer
* Avatar
* Function calling
* Gaming
* stt -> OpenVoice AI
* Monologue System for Inactivity: If there’s no new input from the audience, w-AI-fu triggers a monologue system that introduces new topics, preventing the conversation from going stale.
* prompt with streaming system prompt [] and user: [vedal]: hey... So it can check names and separate from '[chat]
* Need for Standardization – He recognized the need for a standard interface, similar to PCI slots in PCs, to allow seamless integration of different AI models and tools. For diferent tools. And langchain usage?